## Script responsável por descompactar o ZIP

- Antes de executar esse script, fazer download da base do DOU no link: https://www.in.gov.br/acesso-a-informacao/dados-abertos/base-de-dados
- Há um arquivo zip para cada ano/mês/seção
- Salvar no subdiretório \downloads\Secao0<Número da secao>\ano<com 4 dígitos>. Exemplo: \downloads\Secao02\2024
- Processamento deste script:
  - Descompacta todos os arquivos zips existentes no subdiretório downloads\Secao0<Número da secao>
  - Faz a leitura do XML
  - Cria a coluna Texto a partir da TextHTML, removento das tags html do corpo da portaria
  - Cria a coluna Ano com o ano da publicação
  - Cria a coluna Mês com o mês da publicação
  - Cria a coluna Total_palavras, que exibe o total de palavras da coluna Texto
  - Salva a saída no arquivo **.\saida\DouSecao0<No Seção>_completo.parquet**
---


## Bibliotecas

In [1]:
import os
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
import io
import time  # Importa o módulo para medir o tempo

In [2]:
douItem = 2
secao = f'Secao0{douItem}'

In [3]:
def process_zip(zip_path):

    print(zip_path)
    # Lista para armazenar os dados dos artigos
    articles_data = []

    # Abre o arquivo ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Itera sobre cada arquivo no ZIP
        for file_name in zip_ref.namelist():
            if file_name.endswith('.xml'):
                try:
                    # Abre o arquivo XML
                    with zip_ref.open(file_name) as file:
                        # Lê o conteúdo do arquivo
                        content = file.read()
                        # Verifica se o conteúdo não está vazio
                        if content.strip():
                            # Parse do conteúdo XML
                            tree = ET.ElementTree(ET.fromstring(content))
                            root = tree.getroot()

                            # Itera sobre todos os elementos <article>
                            for article in root.findall('article'):
                                if article is not None:
                                    # Extrai os atributos do <article>
                                    article_data = article.attrib

                                    # Extrai os conteúdos dos elementos dentro de <body>
                                    body = article.find('body')
                                    if body is not None:
                                        for child in body:
                                            # Usa o nome do elemento como chave e seu texto como valor
                                            article_data[child.tag] = child.text.strip() if child.text else None

                                    # Adiciona o nome do arquivo aos dados do artigo
                                    article_data['file_name'] = file_name
                                    # Adiciona o nome do arquivo ZIP aos dados do artigo
                                    article_data['zip_name'] = os.path.basename(zip_path)
                                    articles_data.append(article_data)
                except ET.ParseError:
                    print(f"Erro ao analisar XML no arquivo {file_name} no ZIP {zip_path}")
                except Exception as e:
                    print(f"Erro ao processar o arquivo {file_name} no ZIP {zip_path}: {e}")

    return articles_data

In [4]:
def process_all_zips_in_directory(directory):

    print(directory)
    # Lista para armazenar os dados de todos os artigos
    all_articles_data = []

    # Itera sobre todos os arquivos no diretório e subdiretórios
    for root, dirs, files in os.walk(directory):
        print(f'Processando diretório {root}...')
        for file_name in files:
            if file_name.endswith('.zip'):
                zip_path = os.path.join(root, file_name)
                articles_data = process_zip(zip_path)
                all_articles_data.extend(articles_data)

    # Cria um DataFrame a partir dos dados de todos os artigos
    df = pd.DataFrame(all_articles_data)
    return df

In [5]:
# Configuração inicial
douItem = 2
secao = f'Secao0{douItem}'

# Caminho para o diretório contendo os arquivos ZIP
directory_path = os.path.join('./downloads/', secao)

# Marca o início do processamento
start_time = time.time()

# Processa todos os ZIPs no diretório e obtém o DataFrame
df = process_all_zips_in_directory(directory_path)

# Renomeia a coluna Texto
df.rename(columns={'Texto': 'TextoHTML'}, inplace=True)

# Marca o fim do processamento
end_time = time.time()

# Calcula o tempo total de processamento
processing_time = end_time - start_time

# Exibe o DataFrame e o tempo de processamento
print("DataFrame shape:", df.shape)
print(f"Tempo de processamento: {processing_time:.2f} segundos")


./downloads/Secao02
Processando diretório ./downloads/Secao02...
Processando diretório ./downloads/Secao02\2022...
./downloads/Secao02\2022\S02012022.zip
./downloads/Secao02\2022\S02022022.zip
./downloads/Secao02\2022\S02032022.zip
./downloads/Secao02\2022\S02042022.zip
./downloads/Secao02\2022\S02052022.zip
./downloads/Secao02\2022\S02062022.zip
./downloads/Secao02\2022\S02072022.zip
./downloads/Secao02\2022\S02082022.zip
./downloads/Secao02\2022\S02092022.zip
./downloads/Secao02\2022\S02102022.zip
./downloads/Secao02\2022\S02112022.zip
./downloads/Secao02\2022\S02122022.zip
Processando diretório ./downloads/Secao02\2023...
./downloads/Secao02\2023\S02012023.zip
./downloads/Secao02\2023\S02022023.zip
./downloads/Secao02\2023\S02032023.zip
./downloads/Secao02\2023\S02042023.zip
./downloads/Secao02\2023\S02052023.zip
./downloads/Secao02\2023\S02062023.zip
./downloads/Secao02\2023\S02072023.zip
./downloads/Secao02\2023\S02082023.zip
./downloads/Secao02\2023\S02092023.zip
./downloads/Seca

In [6]:
# Caminho do arquivo de saída
output_path = f'./saida/01_DOU{secao}_completo.parquet'

# Marca o início do tempo de escrita
start_time = time.time()

# Salva o arquivo completo em formato Parquet
df.to_parquet(output_path, engine='pyarrow', index=False)

# Marca o fim do tempo de escrita
end_time = time.time()

# Calcula o tempo total de escrita
save_time = end_time - start_time

print(f"Arquivo salvo em: {output_path}")
print(f"Tempo de escrita do arquivo: {save_time:.2f} segundos")


Arquivo salvo em: ./saida/01_DOUSecao02_completo.parquet
Tempo de escrita do arquivo: 11.75 segundos


In [ ]:
df.head()